In [97]:
import pandas as pd

df = pd.read_csv("./property-tax-report-2020-2024.csv", sep=';')

df['CURRENT_PRICE'] = df.apply(lambda x: x['CURRENT_LAND_VALUE']+x['CURRENT_IMPROVEMENT_VALUE'], axis = 1)

df['PREVIOUS_PRICE'] = df.apply(lambda x: x['PREVIOUS_LAND_VALUE']+x['PREVIOUS_IMPROVEMENT_VALUE'], axis = 1)


df = df[['PROPERTY_POSTAL_CODE', 'CURRENT_PRICE', 'PREVIOUS_PRICE', 'REPORT_YEAR', 'YEAR_BUILT']]

print(len(df))

df.to_csv('out.csv', index=False)

1095774


In [98]:
df2 = pd.read_csv("./property-tax-report-2016-2019.csv", sep=';')

df2['CURRENT_PRICE'] = df2.apply(lambda x: x['CURRENT_LAND_VALUE']+x['CURRENT_IMPROVEMENT_VALUE'], axis = 1)

df2['PREVIOUS_PRICE'] = df2.apply(lambda x: x['PREVIOUS_LAND_VALUE']+x['PREVIOUS_IMPROVEMENT_VALUE'], axis = 1)

df2 = df2[['PROPERTY_POSTAL_CODE', 'CURRENT_PRICE', 'PREVIOUS_PRICE', 'REPORT_YEAR', 'YEAR_BUILT']]

In [99]:
df3 = pd.read_csv("./property-tax-report-2011-2015.csv", sep=';')

df3['CURRENT_PRICE'] = df3.apply(lambda x: x['CURRENT_LAND_VALUE']+x['CURRENT_IMPROVEMENT_VALUE'], axis = 1)

df3['PREVIOUS_PRICE'] = df3.apply(lambda x: x['PREVIOUS_LAND_VALUE']+x['PREVIOUS_IMPROVEMENT_VALUE'], axis = 1)

df3 = df3[['PROPERTY_POSTAL_CODE', 'CURRENT_PRICE', 'PREVIOUS_PRICE', 'REPORT_YEAR', 'YEAR_BUILT']]

In [100]:
rldf = pd.read_csv('pccf_record_layout.csv')
# load the raw text pccf file
with open('PCCF_FCCP_V2403_2021.txt','r', encoding='latin-1') as fh:
    pctxt = fh.read()
pclines = pctxt.split('\n')
pclines = list(filter(lambda l: len(l), pclines)) # keep only non-empty lines
print("number of lines:", len(pclines))

number of lines: 1366547


In [101]:
# load the raw text pccf file
with open('pccfNat_fccpNat_052020_1.txt','r', encoding='latin-1') as fh2:
    pctxt2 = fh2.read()
pclines2 = pctxt2.split('\n')
pclines2 = list(filter(lambda l: len(l), pclines2)) # keep only non-empty lines
print("number of lines:", len(pclines2))

number of lines: 1731083


In [102]:
def gen_pccf():
    for idx, r in rldf.iterrows():
        print(idx, r['Field name'])
        
        s = pd.Series(pclines, dtype = "string")
        yield r['Field name'], s.str[r['Position'] - 1 : r['Position'] + r['Size'] - 1]

pccf_df = pd.DataFrame(dict(gen_pccf()))

0 Postal code
1 FSA
2 PR
3 CDuid
4 CSDuid
5 CSDname
6 CSDtype
7 CCScode
8 SAC
9 SACtype
10 CTname
11 ER
12 DPL
13 FED13uid
14 POP_CNTR_RA
15 POP_CNTR_RA_type
16 DAuid
17 Dissemination block
18 Rep_Pt_Type
19 LAT
20 LONG
21 SLI
22 PCtype
23 Comm_Name
24 DMT
25 H_DMT
26 Birth_Date
27 Ret_Date
28 PO
29 QI
30 Source
31 POP_CNTR_RA_SIZE_CLASS


In [103]:
def gen_pccf():
    for idx, r in rldf.iterrows():
        print(idx, r['Field name'])
        
        s = pd.Series(pclines2, dtype = "string")
        yield r['Field name'], s.str[r['Position'] - 1 : r['Position'] + r['Size'] - 1]

pccf_df2 = pd.DataFrame(dict(gen_pccf()))

0 Postal code
1 FSA
2 PR
3 CDuid
4 CSDuid
5 CSDname
6 CSDtype
7 CCScode
8 SAC
9 SACtype
10 CTname
11 ER
12 DPL
13 FED13uid
14 POP_CNTR_RA
15 POP_CNTR_RA_type
16 DAuid
17 Dissemination block
18 Rep_Pt_Type
19 LAT
20 LONG
21 SLI
22 PCtype
23 Comm_Name
24 DMT
25 H_DMT
26 Birth_Date
27 Ret_Date
28 PO
29 QI
30 Source
31 POP_CNTR_RA_SIZE_CLASS


In [105]:
pccf_df['LAT'] = pccf_df['LAT'].astype(float)
pccf_df['LONG'] = pccf_df['LONG'].astype(float)
pccf_short = pccf_df[['Postal code', 'LONG', 'LAT']]
pccf_short = pccf_short.groupby('Postal code').agg({'LONG':'mean','LAT':'mean'})

In [106]:
pccf_df2['LAT'] = pccf_df2['LAT'].astype(float)
pccf_df2['LONG'] = pccf_df2['LONG'].astype(float)
pccf_short2 = pccf_df2[['Postal code', 'LONG', 'LAT', 'CSDname']]
pccf_short2 = pccf_short2.groupby('Postal code').agg({'LONG':'mean','LAT':'mean'})

In [107]:
print(len(pccf_short2))
print(len(pccf_short))

872218
888280


In [108]:
df['Postal code'] = df['PROPERTY_POSTAL_CODE'].str.replace(' ','')
df.drop(columns=['PROPERTY_POSTAL_CODE'])
dfgeo = df.merge(pccf_short, on='Postal code')

In [109]:
df2['Postal code'] = df2['PROPERTY_POSTAL_CODE'].str.replace(' ','')
df2.drop('PROPERTY_POSTAL_CODE', axis = 1)
print(df2.head(10))
dfgeo2 = df2.merge(pccf_short2, on='Postal code')

  PROPERTY_POSTAL_CODE  CURRENT_PRICE  PREVIOUS_PRICE  REPORT_YEAR  \
0              V6N 1J5      4025000.0       4140000.0         2018   
1              V6G 1M1       660000.0        614000.0         2018   
2              V5N 1P2       597000.0        522000.0         2018   
3              V5L 1W8      1163000.0       1172000.0         2018   
4              V5M 1X1      1836000.0       1722000.0         2018   
5              V6R 1Z2      1415000.0       1340000.0         2018   
6              V5W 3G6      1480300.0       1512600.0         2018   
7              V6M 3P2      5547000.0       5496000.0         2018   
8              V6E 1Z8      8591300.0       6574000.0         2018   
9              V6P 1C6      3711200.0       3702200.0         2018   

   YEAR_BUILT Postal code  
0      2009.0      V6N1J5  
1      1985.0      V6G1M1  
2      1991.0      V5N1P2  
3      1911.0      V5L1W8  
4      1948.0      V5M1X1  
5      1989.0      V6R1Z2  
6      1959.0      V5W3G6  
7    

In [110]:
print(len(dfgeo))
print(len(dfgeo2))

1075232
814952


In [111]:
df3['Postal code'] = df3['PROPERTY_POSTAL_CODE'].str.replace(' ','')
df3.drop('PROPERTY_POSTAL_CODE', axis = 1)
print(df3.head(10))
dfgeo3 = df3.merge(pccf_short, on='Postal code')

  PROPERTY_POSTAL_CODE  CURRENT_PRICE  PREVIOUS_PRICE  REPORT_YEAR  \
0              V6P 1Y9      1743000.0             NaN         2013   
1              V6P 2K4      1156000.0             NaN         2013   
2              V6P 2K4      1090600.0             NaN         2013   
3              V6P 2B6      1353400.0             NaN         2013   
4              V6P 2B5      2265000.0             NaN         2013   
5              V6P 2B6      1333400.0             NaN         2013   
6              V6P 2B5      1758000.0             NaN         2013   
7              V6P 2B5      1778000.0             NaN         2013   
8              V6P 2B4      1379400.0             NaN         2013   
9              V6P 2B3      1385700.0             NaN         2013   

   YEAR_BUILT Postal code  
0      2003.0      V6P1Y9  
1      1986.0      V6P2K4  
2      1975.0      V6P2K4  
3      1950.0      V6P2B6  
4      2008.0      V6P2B5  
5      1951.0      V6P2B6  
6      1997.0      V6P2B5  
7    

In [112]:
# print(len(dfgeo))
print(dfgeo.head(10))
print(dfgeo2.head(10))
print(dfgeo3.head(10))
# dfgeo.to_csv('out1.csv', index=False)

  PROPERTY_POSTAL_CODE  CURRENT_PRICE  PREVIOUS_PRICE  REPORT_YEAR  \
0              V6B 1L7       856000.0        829000.0         2023   
1              V6B 1L7       627000.0        607000.0         2023   
2              V6B 1L7       597000.0        567000.0         2022   
3              V6B 1L7       629000.0        631000.0         2022   
4              V6B 1L7       575000.0        563000.0         2022   
5              V6B 1L7       574000.0        588000.0         2022   
6              V6B 1L7       878000.0        853000.0         2023   
7              V6B 1L7       571000.0        559000.0         2022   
8              V6B 1L7       897000.0        862000.0         2024   
9              V6B 1L7       634000.0        617000.0         2024   

   YEAR_BUILT Postal code        LONG        LAT  
0      2005.0      V6B1L7 -123.119508  49.280122  
1      2005.0      V6B1L7 -123.119508  49.280122  
2      2005.0      V6B1L7 -123.119508  49.280122  
3      2005.0      V6B1L7

In [113]:
dfgeo_2024 = dfgeo[(dfgeo['REPORT_YEAR']) == 2024]
dfgeo_2023 = dfgeo[(dfgeo['REPORT_YEAR']) == 2023]
dfgeo_2022 = dfgeo[(dfgeo['REPORT_YEAR']) == 2022]
dfgeo_2021 = dfgeo[(dfgeo['REPORT_YEAR']) == 2021]
dfgeo_2020 = dfgeo[(dfgeo['REPORT_YEAR']) == 2020]
print(len(dfgeo_2024))
print(len(dfgeo_2023))
print(len(dfgeo_2022))
print(len(dfgeo_2021))
print(len(dfgeo_2020))

dfgeo_2024.to_csv('bc_housing_2024.csv', index=False)
dfgeo_2023.to_csv('bc_housing_2023.csv', index=False)
dfgeo_2022.to_csv('bc_housing_2022.csv', index=False)
dfgeo_2021.to_csv('bc_housing_2021.csv', index=False)
dfgeo_2020.to_csv('bc_housing_2020.csv', index=False)

218371
218429
216488
213041
208903


In [114]:
def categorize_price(price):
    if price < 1000000:
        return '< 1M'
    elif 1000000 <= price < 2000000:
        return '1 - 2M'
    elif 2000000 <= price < 5000000:
        return '2 - 5M'
    else:
        return '> 5M'

# Apply the function to create the new 'price_type' column
dfgeo_2024['price_type'] = dfgeo_2024['CURRENT_PRICE'].apply(categorize_price)
dfgeo_2024.to_csv('bc_housing_2024_category_3.csv', index=False)

/var/folders/x6/z9351kk561d4v17pxbj3_rph0000gn/T/ipykernel_29873/3895050187.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfgeo_2024['price_type'] = dfgeo_2024['CURRENT_PRICE'].apply(categorize_price)


In [131]:
value_counts = dfgeo_2024['price_type'].value_counts()

print(value_counts)

< 1M      85647
1 - 2M    64391
2 - 5M    55985
> 5M      12348
Name: price_type, dtype: int64


In [115]:
# dfgeo_2024_geoJson = dfgeo_2024[["CURRENT_PRICE", "LONG", "LAT"]]
# dfgeo_2024_geoJson_5000 = dfgeo_2024_geoJson.iloc[:5000]
# dfgeo_2024_geoJson_5000.to_csv('bc_housing_2024_geoJson_5000.csv', index=False)

In [116]:
# dfgeo_2024 = dfgeo_2024[["CURRENT_PRICE", "LONG", "LAT"]]
# split_index = len(dfgeo_2024) // 2

# # Split the DataFrame
# dfgeo_2024_part1 = dfgeo_2024.iloc[:split_index]
# dfgeo_2024_part2 = dfgeo_2024.iloc[split_index:]

# # Save each part to a separate CSV file
# dfgeo_2024_part1.to_csv('bc_housing_2024_part1.csv', index=False)
# dfgeo_2024_part2.to_csv('bc_housing_2024_part2.csv', index=False)

In [117]:
dfgeo_2019 = dfgeo2[(dfgeo2['REPORT_YEAR']) == 2019]
dfgeo_2018 = dfgeo2[(dfgeo2['REPORT_YEAR']) == 2018]
dfgeo_2017 = dfgeo2[(dfgeo2['REPORT_YEAR']) == 2017]
dfgeo_2016 = dfgeo2[(dfgeo2['REPORT_YEAR']) == 2016]
print(len(dfgeo2))
print(len(dfgeo_2019))
print(len(dfgeo_2018))
print(len(dfgeo_2017))
print(len(dfgeo_2016))

# dfgeo_2019.to_csv('bc_housing_2019.csv', index=False)
# dfgeo_2018.to_csv('bc_housing_2018.csv', index=False)
# dfgeo_2017.to_csv('bc_housing_2017.csv', index=False)
# dfgeo_2016.to_csv('bc_housing_2016.csv', index=False)

814952
209166
204450
202463
198873


In [118]:
dfgeo_2015 = dfgeo3[(dfgeo3['REPORT_YEAR']) == 2015]
print(len(dfgeo_2015))

# dfgeo_2015.to_csv('bc_housing_2015.csv', index=False)

200782


In [119]:
# Assuming dfgeo_2019, dfgeo_2018, dfgeo_2017, dfgeo_2016, dfgeo_2015 are already defined as per your snippet

# Concatenate the DataFrames
df_concatenated = pd.concat([dfgeo_2024, dfgeo_2023, dfgeo_2022, dfgeo_2021, dfgeo_2020, dfgeo_2019, dfgeo_2018, dfgeo_2017, dfgeo_2016, dfgeo_2015])

# Reset the index of the concatenated DataFrame, if needed
df_concatenated.reset_index(drop=True, inplace=True)

# Display the concatenated DataFrame to verify
print(len(df_concatenated))  # To display the first few rows
df_concatenated.to_csv('bc_housing_2015_2024.csv', index=False)

2090966


In [120]:
import pandas as pd

# Provided data
property_count = [203494, 203658, 206480, 209649, 213182, 214803, 217802, 218674, 222197]
year = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

# Convert to DataFrame
df = pd.DataFrame({
    'Year': year,
    'Property Count': property_count
})

# Save to CSV, adjust the file path as needed
csv_file_path = 'property_count_by_year.csv'
df.to_csv(csv_file_path, index=False)

print(f"The data has been saved to {csv_file_path}")


The data has been saved to property_count_by_year.csv


In [121]:
import pandas as pd

# Initial data
property_count = [203494, 203658, 206480, 209649, 213182,  214803, 217802, 218674, 222197, 222298]
year = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

# Convert to DataFrame
df = pd.DataFrame({
    'Year': year,
    'Property Count': property_count
})

# Calculate the percentage increase from the previous year
df['Increased Percentage'] = df['Property Count'].pct_change() * 100
df['Increased Percentage'] = df['Increased Percentage'].fillna(0) # Fill the NaN value for the first year with 0

# Optionally, to round the values for better readability
df['Increased Percentage'] = df['Increased Percentage'].round(2)

# Display the DataFrame to verify
print(df)

# Convert the DataFrame with the new column to a CSV file
csv_file_path = 'property_count_by_year_with_increase.csv'
df.to_csv(csv_file_path, index=False)

print(f"The data has been saved to {csv_file_path}")


   Year  Property Count  Increased Percentage
0  2015          203494                  0.00
1  2016          203658                  0.08
2  2017          206480                  1.39
3  2018          209649                  1.53
4  2019          213182                  1.69
5  2020          214803                  0.76
6  2021          217802                  1.40
7  2022          218674                  0.40
8  2023          222197                  1.61
9  2024          222298                  0.05
The data has been saved to property_count_by_year_with_increase.csv


In [122]:
from geopy.geocoders import Nominatim

def get_city(latitude, longitude):
    geolocator = Nominatim(user_agent="my_unique_application_name")
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    address = location.raw['address']
    city = address.get('city', '')
    if not city:
        city = address.get('town', '')
    if not city:
        city = address.get('village', '')
    return city

# Example usage with coordinates for a place in Vancouver, BC
latitude = 49.259827
longitude = -123.144717
city = get_city(latitude, longitude)
print(city)


Vancouver


In [124]:
# immigration_country_df = pd.read_csv('./immigration_country.csv')
# print(immigration_country_df.columns)

# columns_to_clean = ['year_2015', "year_2016", "year_2017", "year_2018", "year_2019", "year_2020", "year_2021", "year_2022", "year_2023"]

# for column in columns_to_clean:
    
#     immigration_country_df[column] = immigration_country_df[column].str.replace(',', '').fillna(0).astype(int)

# cleaned_file_path = 'immigration_country.csv'
# immigration_country_df.to_csv(cleaned_file_path, index=False)

In [125]:
print(len(immigration_country_df))
print(immigration_country_df.columns)
print(immigration_country_df.head(1))

215
Index(['country_of_citizenship', 'year_2015', 'year_2016', 'year_2017',
       'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022',
       'year_2023'],
      dtype='object')
  country_of_citizenship  year_2015  year_2016  year_2017  year_2018  \
0            Afghanistan       2630       2650       3455       3560   

   year_2019  year_2020  year_2021  year_2022  year_2023  
0       3900       1660       8570      23750      20180  


In [126]:
immigration_bc_df = pd.read_csv('./immigration_bc.csv',  sep=',')

In [127]:
print(len(immigration_bc_df))
print(immigration_bc_df.columns)
print(immigration_bc_df.head(1))

9
Index(['year', 'immigration_count'], dtype='object')
   year  immigration_count
0  2015              35720


In [128]:
from geopy.geocoders import Nominatim

def get_city(latitude, longitude):
    geolocator = Nominatim(user_agent="my_unique_application_name")
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    address = location.raw['address']
    city = address.get('city', '')
    if not city:
        city = address.get('town', '')
    if not city:
        city = address.get('village', '')
    return city

# Example usage with coordinates for a place in Vancouver, BC
latitude = 49.259827
longitude = -123.144717
city = get_city(latitude, longitude)
print(city)


Vancouver


In [129]:
import pandas as pd

# Initial data
property_count = [203494, 203658, 206480, 209649, 213182,  214803, 217802, 218674, 222197]
year = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

# Convert to DataFrame
df = pd.DataFrame({
    'Year': year,
    'Property Count': property_count
})

# Calculate the percentage increase from the previous year
df['Increased Percentage'] = df['Property Count'].pct_change() * 100
df['Increased Percentage'] = df['Increased Percentage'].fillna(0) # Fill the NaN value for the first year with 0

# Optionally, to round the values for better readability
df['Increased Percentage'] = df['Increased Percentage'].round(2)

# Display the DataFrame to verify
print(df)

# Convert the DataFrame with the new column to a CSV file
csv_file_path = 'immigration_bc.csv'
df.to_csv(csv_file_path, index=False)

print(f"The data has been saved to {csv_file_path}")


   Year  Property Count  Increased Percentage
0  2015          203494                  0.00
1  2016          203658                  0.08
2  2017          206480                  1.39
3  2018          209649                  1.53
4  2019          213182                  1.69
5  2020          214803                  0.76
6  2021          217802                  1.40
7  2022          218674                  0.40
8  2023          222197                  1.61
The data has been saved to immigration_bc.csv


In [130]:
import pandas as pd

# Initial data
total_immigration_number = [271840, 296375, 286535, 321055, 341175,  184596, 406050, 437625, 471170]
year = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

# Convert to DataFrame
df = pd.DataFrame({
    'Year': year,
    'total_immigration_number': total_immigration_number
})

# Display the DataFrame to verify
print(df)

# Convert the DataFrame with the new column to a CSV file
csv_file_path = 'immigration_total_count.csv'
df.to_csv(csv_file_path, index=False)

print(f"The data has been saved to {csv_file_path}")


   Year  total_immigration_number
0  2015                    271840
1  2016                    296375
2  2017                    286535
3  2018                    321055
4  2019                    341175
5  2020                    184596
6  2021                    406050
7  2022                    437625
8  2023                    471170
The data has been saved to immigration_total_count.csv
